In [3]:
import pandas as pd # Data processing, CSV file I/O 
import numpy as np # Linear Algebra
from sklearn.model_selection import train_test_split
from keras.models import Sequential
# The various layers for the Neural Network Model.
# Dense - A layer that is fully connected (densely-connected.)
# Conv2D - A 2-dimensional convolutional layer.
# Dropout - A layer that helps prevent overfitting.
# Flatten - A layer that flattens the input.
# MaxPooling2D - A layer that performs Max Pooling of the Convolutions
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Since I have a GPU & I've GPU enabled, I am going to use the GPU version of keras 
# (NOTE: Ignore if you do not have GPU enabled)
import tensorflow as tf
from keras import backend as K
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
#K.tensorflow_backend._get_available_gpus()

modelLoaded = False

Using TensorFlow backend.


In [4]:
#dataset = pd.read_csv(filepath_or_buffer="data.csv", 
#                      sep=",")

#trainSet, testSet = train_test_split(dataset,test_size = 0.2)

#trainSet.to_pickle("train_set.pkl")
#testSet.to_pickle("test_set.pkl")

#print(dataset)

In [120]:
trainSet = pd.read_pickle("train_set.pkl")
testSet = pd.read_pickle("test_set.pkl")

In [10]:
model = tf.keras.models.load_model("Models\\CNN_01.model")
modelLoaded = True

In [5]:
###Divisione ed elaborazione del dataset

In [121]:
trainSet_x = trainSet.drop("Label",axis = 1).div(255.0)
trainSet_label = trainSet["Label"]

testSet_x = testSet.drop("Label",axis = 1).div(255.0)
testSet_label = testSet["Label"]

Labels = trainSet_label.unique()
i=0
intToLabel = {}
labelToInt = {}
for l in Labels:
    intToLabel[i] = str(l)
    labelToInt[l] = i
    i+=1

f= open("IndexChar.txt","w")
for i in range(len(intToLabel)):
    f.write(str(i) + " " + intToLabel[i] + "\n")
f.close()
    
trainSet_x = trainSet_x.values.reshape(trainSet.shape[0],28,28,1)
testSet_x = testSet_x.values.reshape(testSet.shape[0],28,28,1)

trainSet_y = []
for l in trainSet_label:
    trainSet_y.append(labelToInt[l])

testSet_y = []
for l in testSet_label:
    testSet_y.append(labelToInt[l])
    
print(len(Labels))

84


In [5]:
#print(Labels)

In [6]:
###

In [7]:
###Creazione e training del modello

In [8]:
modelLoaded = False
INPUT_SHAPE = trainSet_x[0].shape
BATCH_SIZE = 128
EPOCHS = 10
TOT_LABELS = max(trainSet_y)+1

In [9]:
if not modelLoaded:
    model = Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same', input_shape=INPUT_SHAPE))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same', input_shape=INPUT_SHAPE))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    
    #model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dense(TOT_LABELS, activation='softmax'))
    
    print (model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               25690624  
_________________________________________________________________
dense_2 (Dense)              (None, 84)                43092     
Total params: 25,789,460
Trainable params: 25,789,460
Non-trainable pa

model.summary()

In [10]:
if not modelLoaded:
    model.compile(optimizer="adam",
                 loss="sparse_categorical_crossentropy",
                 metrics=["accuracy"])
    model.fit(x=trainSet_x,
              y=trainSet_y,
              epochs=EPOCHS,
              batch_size=BATCH_SIZE,
              validation_split=0.1)

Instructions for updating:
Use tf.cast instead.
Train on 270701 samples, validate on 30078 samples
Epoch 1/10
270701/270701 [==============================] - 4334s 16ms/step - loss: 0.2910 - acc: 0.9211 - val_loss: 0.1082 - val_acc: 0.9679
Epoch 2/10
270701/270701 [==============================] - 4287s 16ms/step - loss: 0.0730 - acc: 0.9778 - val_loss: 0.0660 - val_acc: 0.9815
Epoch 3/10
270701/270701 [==============================] - 4343s 16ms/step - loss: 0.0456 - acc: 0.9855 - val_loss: 0.0538 - val_acc: 0.9859
Epoch 4/10
270701/270701 [==============================] - 4431s 16ms/step - loss: 0.0332 - acc: 0.9893 - val_loss: 0.0511 - val_acc: 0.9871
Epoch 5/10
270701/270701 [==============================] - 4420s 16ms/step - loss: 0.0275 - acc: 0.9911 - val_loss: 0.0467 - val_acc: 0.9898
Epoch 6/10
270701/270701 [==============================] - 4408s 16ms/step - loss: 0.0241 - acc: 0.9925 - val_loss: 0.0484 - val_acc: 0.9891
Epoch 7/10
270701/270701 [=======================

In [11]:
if not modelLoaded:
    model.save("CNN.model")

In [ ]:
###

In [ ]:
###Test

In [ ]:
print(testSet_x)
print(testSet_y)

In [12]:
value = model.evaluate(testSet_x, testSet_y)

print(model.metrics_names)
print(value)

75195/75195 [==============================] - 357s 5ms/step
['loss', 'acc']
[0.043881907415588214, 0.9904248952802187]


In [11]:
predictions = model.predict(testSet_x)

In [15]:
correct = 0
i=0
for y in testSet_label:
    p = np.argmax(predictions[i])
    #print(str(y) + " - " + intToLabel[p])
    if intToLabel[p] == str(y):
        correct += 1
    i+=1
 
print((correct/len(predictions))*100)

95.53028791807967


In [126]:
def ReadIndexChar(path):
    f= open(path,"r")
    lines = f.readlines()
    intToLabels = {}
    for l in lines:
        number = int(l.split(" ")[0])
        char = l.split(" ")[1][0]
        intToLabels[number] = char
    return intToLabels

In [129]:
ItoC = ReadIndexChar("IndexChar.txt")
print(ItoC)

{0: 'X', 1: '-', 2: '+', 3: '1', 4: 'e', 5: '4', 6: 'A', 7: 'i', 8: 'i', 9: 't', 10: 's', 11: ')', 12: 'N', 13: 'g', 14: '9', 15: 'v', 16: 'd', 17: 'p', 18: '=', 19: 'z', 20: '(', 21: 'f', 22: 'G', 23: '2', 24: 'b', 25: 'u', 26: '8', 27: '3', 28: '7', 29: 'C', 30: '0', 31: 'a', 32: 'y', 33: 'c', 34: 'R', 35: ']', 36: 'S', 37: 'b', 38: 'M', 39: 'i', 40: 'q', 41: 'l', 42: 't', 43: 's', 44: 'j', 45: ',', 46: 'l', 47: 'l', 48: '6', 49: 'a', 50: 'H', 51: '5', 52: '!', 53: 'p', 54: 'T', 55: 'l', 56: 'k', 57: '}', 58: 'p', 59: 'r', 60: 'w', 61: 'n', 62: 'l', 63: 'l', 64: 't', 65: '9', 66: 's', 67: 'l', 68: 'p', 69: 'g', 70: 'o', 71: '{', 72: 'd', 73: 'g', 74: 'p', 75: '[', 76: 'm', 77: 'e', 78: '0', 79: 'D', 80: 'f', 81: 's', 82: 'f', 83: 'i'}


In [132]:
def Classify (model, input, intToLabels, numberProbabilities = 3):
    predictions = model.predict(np.array( [input,]))
    labelProbabilities = []
    for i in range(len(predictions[0])):
        labelProbabilities.append((intToLabels[i],predictions[0][i]))
    #a = np.array(labelProbabilities)
    
    dtype = [('label', str), ('probabilities', float)]
    a = np.array(labelProbabilities, dtype=dtype)       # create a structured array
    a = np.sort(a, order='probabilities')   
    a = a[::-1]
    #labelProbabilities.sort()
    return a[:numberProbabilities]



In [133]:
print(classify(model, testSet_x[0], ItoC))

[(b'+', 1.00000000e+00) (b'a', 3.25442393e-08) (b'-', 1.07836329e-09)]
